<a href="https://colab.research.google.com/github/marmurr/Deep-Learning/blob/main/Deep_Learning_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pytorch.org/tutorials/

Написати свій MLP class:
можна конфігурувати

кількість hidden layers

розміри hidden layers

Використовувані активації для кожного layer

клас має бути написаний саме в PyTorch стилі

Взяти модельний/сподобавшийся датасет (але тільки класифікація) (можна sklearn) та побудувати

Dataset

Dataloader

Training loop



1. Import

In [1]:
import torch
import matplotlib.pyplot as plt

2. MLP

In [2]:
import torch.nn as nn

In [3]:
class CustomLayer(nn.Module):
    def __init__(self, inputs, outputs, activation=nn.ReLU, batchnorm=None, p=None):
        super(CustomLayer, self).__init__()
        self.linear = nn.Linear(inputs, outputs)
        self.activation = activation if activation else nn.Identity()
        self.batchnorm = nn.BatchNorm1d(outputs) if batchnorm else None
        self.dropout = nn.Dropout(p) if p else None

    def forward(self, x):
        x = self.linear(x)
        x = self.activation(x)
        if self.batchnorm is not None:
            x = self.batchnorm(x)
        if self.dropout is not None:
            x = self.dropout(x)
        return x

class CustomMLP(nn.Module):
    def __init__(self, layer_sizes, activation_funcs=None, batchnorm_list=None, dropout_list=None):
        super(CustomMLP, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layer_sizes) - 1):
            activation = activation_funcs[i] if activation_funcs and i < len(activation_funcs) else nn.Identity()
            batchnorm = batchnorm_list[i] if batchnorm_list and i < len(batchnorm_list) else None
            p = dropout_list[i] if dropout_list and i < len(dropout_list) else None
            layer = CustomLayer(layer_sizes[i], layer_sizes[i+1], activation, p=p, batchnorm=batchnorm)
            self.layers.append(layer)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


3. Test


In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim

In [5]:
#Data loading and normalizing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

100%|██████████| 26421880/26421880 [00:01<00:00, 14405425.19it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 272907.38it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 4881801.96it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 2844082.85it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



Model initializing

Our dataset has 10 classes and 28x28 grayscale images

In [6]:
import torch.nn.functional as F

In [7]:
model = CustomMLP(layer_sizes=[784, 256, 128, 10], activation_funcs=[F.relu, F.relu])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

Model training

In [8]:
epochs = 10
for epoch in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        images = images.view(images.shape[0], -1)
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1} - Training loss: {running_loss/len(train_loader)}")

Epoch 1 - Training loss: 0.505927483727937
Epoch 2 - Training loss: 0.38433797937085123
Epoch 3 - Training loss: 0.3553953744899998
Epoch 4 - Training loss: 0.3350270961774692
Epoch 5 - Training loss: 0.3208127060869355
Epoch 6 - Training loss: 0.30756989366083004
Epoch 7 - Training loss: 0.29507604698890816
Epoch 8 - Training loss: 0.285487912912994
Epoch 9 - Training loss: 0.27883236469236267
Epoch 10 - Training loss: 0.2751006612867943
